### Setup

In [1]:
# Import libraries
# AWS:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

# MeteoStat
!pip install meteostat
!pip install pyarrow
!pip install fastparquet
from meteostat import Stations
from datetime import datetime
from meteostat import Monthly
from meteostat import Hourly

# Elexon
!pip install ElexonDataPortal
from ElexonDataPortal import api
client = api.Client('gj9exjxj042zjbs')

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# This line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " 
      + xgboost_container + " container for your SageMaker endpoint.")

Success - the MySageMakerInstance is in the eu-west-2 region. You will use the 644912444149.dkr.ecr.eu-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [3]:
bucket_name = 'this-is-a-valid-bucket-name' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


### MeteoStat API Data

In [4]:
# Select weather stations in Great Britain
stations = Stations()
stations = stations.region('GB')
print('Stations in Great Britain:', stations.count())

Stations in Great Britain: 177


In [5]:
# Fetch class data
stations.fetch()

,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end
id,,,,,,,,,,,,,,,
03003,Sumburgh Cape,GB,SCT,03003,EGPB,59.8833,-1.3000,5.0,Europe/London,1977-01-19,2022-08-08,1978-02-25,2022-04-25,1983-01-01,2022-01-01
03005,Lerwick,GB,ZET,03005,<NA>,60.1333,-1.1833,82.0,Europe/London,2018-01-27,2022-08-08,1929-10-01,2022-08-02,1929-01-01,2022-01-01
03014,Foula,GB,ZET,03014,<NA>,60.1167,-2.0667,13.0,Europe/London,NaT,NaT,1984-08-01,2016-04-26,1984-01-01,2016-01-01
03017,Kirkwall Airport,GB,SCT,03017,EGPA,58.9500,-2.9000,21.0,Europe/London,1973-01-01,2022-08-08,1973-01-01,2022-08-02,1973-01-01,2022-01-01
03022,Benbecula,GB,SCT,03022,EGPL,57.4667,-7.3667,6.0,Europe/London,1973-01-01,2022-08-08,1973-01-03,2022-04-25,1982-01-01,2022-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EGUO0,Colerne,GB,ENG,<NA>,EGUO,51.4391,-2.2864,180.0,Europe/London,1942-04-28,2019-02-14,1943-07-27,1947-03-30,1944-01-01,1946-01-01
EGVI0,Greenham Common / Shaw,GB,ENG,<NA>,EGVI,51.3833,-1.2833,122.0,Europe/London,1976-04-28,1992-04-17,1976-08-31,1989-11-14,NaT,NaT
EGVT0,Wethersfield,GB,ENG,<NA>,EGVT,51.9667,0.5000,101.0,Europe/London,1989-03-04,1990-01-22,NaT,NaT,NaT,NaT


In [6]:
# Select station IDs
station_ids = stations.fetch().index.values

# Select city names
station_city = stations.fetch()['name'].values

# Create dictionary to match station IDs to city names
station_dict = dict(zip(station_ids, station_city))

# Create list of station IDs
station_ids = station_ids.tolist()

In [7]:
# Set date parameters for 2021
start = datetime(2021, 1, 1)
end = datetime(2022, 1, 1, 23, 59)

# Set empty dataframe
df1 = pd.DataFrame()

# Iterate through stations and concatenate data to dataframe
for station in station_ids:
    data = Hourly(station, start, end)
    data = data.fetch()
    data['station'] = station
    df1 = pd.concat([df1, data])

In [8]:
# Set date parameters for 2020
start = datetime(2020, 1, 1)
end = datetime(2021, 1, 1, 23, 59)

# Set empty dataframe
df2 = pd.DataFrame()

# Iterate through stations and concatenate data to dataframe
for station in station_ids:
    data = Hourly(station, start, end)
    data = data.fetch()
    data['station'] = station
    df2 = pd.concat([df2, data])

In [9]:
# Concatenate both dataframes
df_final = pd.concat([df1, df2])
df_final.sort_values(by='time', inplace=True)
df_final

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,station,date,hour
time,,,,,,,,,,,,,,
2020-01-01 00:00:00,5.0,3.7,91.0,NaN,NaN,80.0,16.6,NaN,1031.0,NaN,NaN,03214,NaN,NaN
2020-01-01 00:00:00,6.0,4.0,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03644,NaN,NaN
2020-01-01 00:00:00,3.5,1.5,87.0,NaN,NaN,218.0,7.4,16.7,1027.7,NaN,4.0,03114,NaN,NaN
2020-01-01 00:00:00,3.5,3.2,98.0,NaN,NaN,160.0,9.4,14.8,1032.0,NaN,5.0,03257,NaN,NaN
2020-01-01 00:00:00,5.0,4.6,97.0,NaN,NaN,130.0,18.4,NaN,1032.5,NaN,NaN,03392,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01 23:00:00,12.5,10.4,87.0,0.0,NaN,190.0,14.8,NaN,1013.1,NaN,NaN,03658,NaN,NaN
2022-01-01 23:00:00,12.0,10.1,88.0,NaN,NaN,190.0,11.2,18.5,1011.0,NaN,7.0,03715,NaN,NaN
2022-01-01 23:00:00,9.8,8.9,94.0,0.2,NaN,200.0,14.8,NaN,1007.6,NaN,NaN,03214,NaN,NaN


In [10]:
df_final.info()
df_final.describe()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3745308 entries, 2020-01-01 00:00:00 to 2022-01-01 23:00:00
Data columns (total 14 columns):
 #   Column   Dtype  
---  ------   -----  
 0   temp     float64
 1   dwpt     float64
 2   rhum     float64
 3   prcp     float64
 4   snow     float64
 5   wdir     float64
 6   wspd     float64
 7   wpgt     float64
 8   pres     float64
 9   tsun     float64
 10  coco     float64
 11  station  object 
 12  date     object 
 13  hour     object 
dtypes: float64(11), object(3)
memory usage: 428.6+ MB


,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
count,3.741386e+06,3.739929e+06,3.739999e+06,506925.000000,18756.000000,3.709961e+06,3.720725e+06,2.465183e+06,3.722990e+06,436.000000,2.578145e+06
mean,1.030640e+01,7.031784e+00,8.161255e+01,0.111507,13.404777,1.953121e+02,1.781366e+01,2.917136e+01,1.013366e+03,10.334862,4.741020e+00
std,5.489455e+00,4.984771e+00,1.383226e+01,0.770948,106.853740,9.390784e+01,1.144542e+01,1.490650e+01,1.269400e+01,19.788021,3.649981e+00
min,-1.990000e+01,-4.470000e+01,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,9.477000e+02,0.000000,0.000000e+00
25%,6.500000e+00,3.400000e+00,7.400000e+01,0.000000,0.000000,1.300000e+02,9.400000e+00,1.850000e+01,1.005700e+03,0.000000,2.000000e+00
50%,1.000000e+01,7.100000e+00,8.500000e+01,0.000000,0.000000,2.100000e+02,1.500000e+01,2.590000e+01,1.015000e+03,0.000000,4.000000e+00
75%,1.400000e+01,1.090000e+01,9.300000e+01,0.000000,10.000000,2.600000e+02,2.410000e+01,3.700000e+01,1.022000e+03,6.000000,7.000000e+00
max,3.690000e+01,2.250000e+01,1.040000e+02,256.200000,9960.000000,3.600000e+02,3.450000e+02,1.500000e+02,1.062900e+03,60.000000,2.700000e+01


In [11]:
# Drop 'snow', wpgt', 'date' and 'hour' columns due to number of NaNs
df_final.drop('snow', axis = 1, inplace = True)
df_final.drop('wpgt', axis = 1, inplace = True)
df_final.drop('tsun', axis = 1, inplace = True)
df_final.drop('coco', axis = 1, inplace = True)
df_final.drop('date', axis = 1, inplace = True)
df_final.drop('hour', axis = 1, inplace = True)

In [12]:
# Map dictionary to a new column
df_final['city'] = df_final['station'].map(station_dict)

# Reset index
df_final = df_final.reset_index().copy()

# Create 'ID' column
df_final['id'] = [i for i in range(len(df_final))]

# Modify structure of columns
df_final.insert(0, "datetime", df_final.pop("time"))
df_final.drop('city', axis=1)

,datetime,temp,dwpt,rhum,prcp,wdir,wspd,pres,station,id
0,2020-01-01 00:00:00,5.0,3.7,91.0,NaN,80.0,16.6,1031.0,03214,0
1,2020-01-01 00:00:00,6.0,4.0,87.0,NaN,NaN,NaN,NaN,03644,1
2,2020-01-01 00:00:00,3.5,1.5,87.0,NaN,218.0,7.4,1027.7,03114,2
3,2020-01-01 00:00:00,3.5,3.2,98.0,NaN,160.0,9.4,1032.0,03257,3
4,2020-01-01 00:00:00,5.0,4.6,97.0,NaN,130.0,18.4,1032.5,03392,4
...,...,...,...,...,...,...,...,...,...,...
3745303,2022-01-01 23:00:00,12.5,10.4,87.0,0.0,190.0,14.8,1013.1,03658,3745303
3745304,2022-01-01 23:00:00,12.0,10.1,88.0,NaN,190.0,11.2,1011.0,03715,3745304
3745305,2022-01-01 23:00:00,9.8,8.9,94.0,0.2,200.0,14.8,1007.6,03214,3745305
3745306,2022-01-01 23:00:00,12.0,11.8,99.0,NaN,190.0,25.9,1011.8,03809,3745306


In [13]:
df_final

,datetime,temp,dwpt,rhum,prcp,wdir,wspd,pres,station,city,id
0,2020-01-01 00:00:00,5.0,3.7,91.0,NaN,80.0,16.6,1031.0,03214,Walney Island,0
1,2020-01-01 00:00:00,6.0,4.0,87.0,NaN,NaN,NaN,NaN,03644,Fairford Royal Air Force Base,1
2,2020-01-01 00:00:00,3.5,1.5,87.0,NaN,218.0,7.4,1027.7,03114,Oban,2
3,2020-01-01 00:00:00,3.5,3.2,98.0,NaN,160.0,9.4,1032.0,03257,Leeming,3
4,2020-01-01 00:00:00,5.0,4.6,97.0,NaN,130.0,18.4,1032.5,03392,Wainfleet,4
...,...,...,...,...,...,...,...,...,...,...,...
3745303,2022-01-01 23:00:00,12.5,10.4,87.0,0.0,190.0,14.8,1013.1,03658,Benson,3745303
3745304,2022-01-01 23:00:00,12.0,10.1,88.0,NaN,190.0,11.2,1011.0,03715,Cardiff-Wales Airport,3745304
3745305,2022-01-01 23:00:00,9.8,8.9,94.0,0.2,200.0,14.8,1007.6,03214,Walney Island,3745305
3745306,2022-01-01 23:00:00,12.0,11.8,99.0,NaN,190.0,25.9,1011.8,03809,Culdrose,3745306


In [14]:
df_jk_this_is_final = df_final.groupby(['datetime']).mean()
df_jk_this_is_final

,temp,dwpt,rhum,prcp,wdir,wspd,pres,id
datetime,,,,,,,,
2020-01-01 00:00:00,5.079508,3.476860,90.066116,NaN,140.591304,13.130435,1029.321667,121.5
2020-01-01 01:00:00,5.016031,3.514615,90.746154,NaN,145.374046,13.293893,1028.759836,374.5
2020-01-01 02:00:00,4.997710,3.501538,90.707692,NaN,145.546154,12.825191,1028.256911,636.5
2020-01-01 03:00:00,5.132061,3.630769,90.707692,NaN,152.534884,12.830534,1027.871901,898.5
2020-01-01 04:00:00,5.177863,3.700769,90.784615,NaN,153.852713,12.929771,1027.515702,1160.5
...,...,...,...,...,...,...,...,...
2022-01-01 19:00:00,11.391333,9.143333,86.293333,0.061290,203.426667,25.971333,1008.995333,3744632.5
2022-01-01 20:00:00,11.148667,9.035333,87.033333,0.053226,203.440000,24.844667,1008.976000,3744782.5
2022-01-01 21:00:00,10.960667,8.900667,87.326667,0.051613,204.573333,24.698000,1008.912000,3744932.5


In [19]:
df_jk_this_is_final.drop('id', axis=1, inplace=True)

In [21]:
df_jk_this_is_final.to_parquet("weather.parquet", compression = "gzip")

### Elexon API Data

Demand:  
B0610 - Actual Total Load  

Balancing:  
B1770 - Imbalance Prices  
B1780 - Aggregated Imbalance Volumes  

Generation:  
B1430 - Day-Ahead Aggregated Generation Forecast  
B1440 - Generation Forecasts For Wind And Solar  
B1630 - Actual Or Estimated Wind And Solar Power Generation  

### Actual Total Load

In [17]:
# Actual Load of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
AL21 = client.get_B0610(start_date, end_date)

B0610:   6%|▋         | 1109/17520 [38:19<9:27:11,  2.07s/it] 


ConnectionError: HTTPSConnectionPool(host='api.bmreports.com', port=443): Max retries exceeded with url: /BMRS/B0610/v1?APIKey=gj9exjxj042zjbs&SettlementDate=2021-01-24&Period=6&ServiceType=xml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbaa519ae80>: Failed to establish a new connection: [Errno 110] Connection timed out'))

In [ ]:
# save file
df_AL21 = AL21[['local_datetime','settlementPeriod','quantity']]
df_AL21.to_csv('Actual_Load_2021.csv',index=False)

In [ ]:
# Actual Load of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
AL20 = client.get_B0610(start_date, end_date)

In [ ]:
# save file
df_AL20 = AL20[['local_datetime','settlementPeriod','quantity']]
df_AL20 = df_ALS20.drop_duplicates(subset=['local_datetime', 'settlementPeriod'], keep='first')
df_AL20.to_csv('Actual_Load_2020.csv',index=False)

### Imbalance Prices

In [ ]:
# Imbalance Price of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
IP21 = client.get_B1770(start_date, end_date)

In [ ]:
# delete useless columns
df_IP21 = IP21[['local_datetime','settlementPeriod','imbalancePriceAmountGBP']].drop_duplicates().reset_index(drop=True)

# save file
df_IP21.to_csv('Imbalance_Price_2021.csv',index=False)

In [ ]:
# Imbalance Price of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
IP20 = client.get_B1770(start_date, end_date)

In [ ]:
# delete useless columns
df_IP20 = IP20[['local_datetime','settlementPeriod','imbalancePriceAmountGBP']].drop_duplicates().reset_index(drop=True)

# save file
df_IP20.to_csv('Imbalance_Price_2020.csv',index=False)

### Aggregate Imbalance Volumes

In [ ]:
# Imbalance Volume of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
IV21 = client.get_B1780(start_date, end_date)

In [ ]:
# save file
df_IV21 = IV21[['local_datetime','settlementPeriod','imbalanceQuantityMAW']]
df_IV21 = df_IV21.drop_duplicates(subset=['local_datetime', 'settlementPeriod'], keep='first')
df_IV21.to_csv('Imbalance_Volume_2021.csv',index=False)

In [ ]:
# Imbalance Volume of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
IV20 = client.get_B1780(start_date, end_date)

In [ ]:
# save file
df_IV20 = IV20[['local_datetime','settlementPeriod','imbalanceQuantityMAW']]
df_IV20.to_csv('Imbalance_Volume_2020.csv',index=False)

### Day-Ahead Aggregated Generation Forecast

In [ ]:
# Day-Ahead Aggregated Generation Forecast of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
GF21 = client.get_B1430(start_date, end_date)

In [ ]:
# save file
df_GF21 = GF21[['local_datetime','settelmentPeriod','quantity']]
df_GF21.to_csv('DA_Aggregated_Generation_Forecast_2021.csv',index=False)

In [ ]:
# Day-Ahead Aggregated Generation Forecast of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
GF20 = client.get_B1430(start_date, end_date)

In [ ]:
# save file
df_GF20 = GF20[['local_datetime','settelmentPeriod','quantity']]
df_GF20.to_csv('DA_Aggregated_Generation_Forecast_2020.csv',index=False)

### Generation Forecasts For Wind And Solar

In [ ]:
# Wind and Solar Generation Forecast of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
WSGF21 = client.get_B1440(start_date, end_date)

In [ ]:
# save file
df1 = WSGF21[WSGF21['processType']=='Day Ahead']
SF21 = df1[df1['powerSystemResourceType']=='"Solar"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
WOFF21 = df1[df1['powerSystemResourceType']=='"Wind Offshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
WONF21 = df1[df1['powerSystemResourceType']=='"Wind Onshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)

SF21.to_csv('DA_Solar_Generation_Forecast_2021.csv',index=False)
WOFF21.to_csv('DA_Wind_Offshore_Generation_Forecast_2021.csv',index=False)
WONF21.to_csv('DA_Wind_Onshore_Generation_Forecast_2021.csv',index=False)

In [ ]:
# Wind and Solar Generation Forecast of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
WSGF20 = client.get_B1440(start_date, end_date)

In [ ]:
# save file
df1 = WSGF20[WSGF20['processType']=='Day Ahead']
SF20 = df1[df1['powerSystemResourceType']=='"Solar"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
WOFF20 = df1[df1['powerSystemResourceType']=='"Wind Offshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
WONF20 = df1[df1['powerSystemResourceType']=='"Wind Onshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)

SF20.to_csv('DA_Solar_Generation_Forecast_2020.csv',index=False)
WOFF20.to_csv('DA_Wind_Offshore_Generation_Forecast_2020.csv',index=False)
WONF20.to_csv('DA_Wind_Onshore_Generation_Forecast_2020.csv',index=False)

### Actual Generation for Wind And Solar

In [ ]:
# Actual Wind and Solar Generation of 2021
start_date = '2021-01-01'
end_date = '2022-01-01'
AWSG21 = client.get_B1630(start_date, end_date)

In [ ]:
# save file
AWSG21 = AWSG21[AWSG21['powerSystemResourceType']=='"Solar"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
AWOFG21 = AWS21[AWS21['powerSystemResourceType']=='"Wind Offshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
AWONG21 = AWS21[AWS21['powerSystemResourceType']=='"Wind Onshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)

ASG21.to_csv('Actual_Solar_Generation_2021.csv',index=False)
AWOFG21.to_csv('Actual_Wind_Offshore_Generation_2021.csv',index=False)
AWONG21.to_csv('Actual_Wind_Onshore_Generation_2021.csv',index=False)

In [ ]:
# Actual Wind and Solar Generation of 2020
start_date = '2020-01-01'
end_date = '2021-01-01'
AWSG20 = client.get_B1630(start_date, end_date)

In [ ]:
# save file
AWSG20 = AWSG20[AWSG20['powerSystemResourceType']=='"Solar"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
AWOFG20 = AWS20[AWS20['powerSystemResourceType']=='"Wind Offshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)
AWONG20 = AWS20[AWS20['powerSystemResourceType']=='"Wind Onshore"'][['local_datetime','settlementPeriod','quantity']].reset_index(drop=True)

ASG20.to_csv('Actual_Solar_Generation_2020.csv',index=False)
AWOFG20.to_csv('Actual_Wind_Offshore_Generation_2020.csv',index=False)
AWONG20.to_csv('Actual_Wind_Onshore_Generation_2020.csv',index=False)

### Merging Elexon API Data

### 2021

In [ ]:
dfs_2021 = [df_AL21, df_IP21, df_IV21, df_GF21, SF21, WOFF21, WONF21, ASG21, AWOFG21, AWONG21]
dfs_2021_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='inner'), dfs_2021)
dfs_2021_merged

In [ ]:
dfs_2021_merged.columns = ['Datetime','SettlementPeriod','Actual Load', 'Imbalance Price', 'Imbalance Volume', 
                           'Generation Forecast', 'Solar  Forecast', 'Wind Offshore Forecast', 'Wind Onshore Forecast',
                           'Actual Solar Generation', 'Actual Wind Offshore Generation', 'Actual Wind Onshore Generation']
dfs_2021_merged['datetime'] = dfs_2021_merged['datetime'].str[:-6]
dfs_2021_merged.drop('settlementPeriod', axis=1, inplace=True)
dfs_2021_merged.head()

### 2020

In [ ]:
dfs_2020 = [df_AL20, df_IP20, df_IV20, df_GF20, SF20, WOFF20, WONF20, ASG20, AWOFG20, AWONG20]
dfs_2020_merged = reduce(lambda left,right: pd.merge(left,right,on=['local_datetime','settlementPeriod'],how='inner'), dfs_2020)
dfs_2020_merged

In [ ]:
dfs_2020_merged.columns = ['Datetime','SettlementPeriod','Actual Load', 'Imbalance Price', 'Imbalance Volume', 
                           'Generation Forecast', 'Solar  Forecast', 'Wind Offshore Forecast', 'Wind Onshore Forecast',
                           'Actual Solar Generation', 'Actual Wind Offshore Generation', 'Actual Wind Onshore Generation']
dfs_2020_merged['datetime'] = dfs_2020_merged['datetime'].str[:-6]
dfs_2020_merged.drop('settlementPeriod', axis=1, inplace=True)
dfs_2020_merged.head()

### Merging 2021 & 2020

In [ ]:
df_merged = pd.concat([df_merged_2021, df_merged_2020]).reset_index(drop=True)
df_merged

### Merging MeteoStat and Elexon Data

### Model Training

The following two sections are copied from https://aws.amazon.com/getting-started/hands-on/build-train-deploy-machine-learning-model-sagemaker/ and represent the model training and evaluation path that could be used with the pre-built AWS SageMaker XGBoost Algorithm

In [ ]:
train_data, test_data = np.split(df_final_2.sample(frac=1, random_state=3333), [int(0.7 * len(df_final_2))])
print(train_data.shape, test_data.shape)

In [ ]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [ ]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

In [ ]:
xgb.fit({'train': s3_input_train})

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
from sagemaker.serializers import CSVSerializer

test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

### Model Performance Evaluation

In [ ]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))